# 🔥 AI-Based Forest Fire & Smoke Detection

---

## Project Overview

This notebook implements a **comprehensive machine learning pipeline** for forest fire and smoke detection.

**Two Approaches for Handling Class Imbalance:**
1. **Without SMOTE**: Gaussian/Multivariate Normal anomaly detection
2. **With SMOTE**: Traditional classifiers (RF, GB, AdaBoost, etc.)

**Author:** Avirup Roy | **Date:** February 2026 | **Version:** 2.1.0

---

## 1. Environment Setup & Imports

In [ ]:
# Standard Library
import random, sys, json, warnings
warnings.filterwarnings('ignore')

# Data Manipulation
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import ttest_ind, mannwhitneyu, shapiro, multivariate_normal

# Machine Learning
import sklearn
from sklearn.model_selection import (train_test_split, StratifiedKFold, cross_val_score,
    cross_validate, GridSearchCV, RandomizedSearchCV)
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.ensemble import (RandomForestClassifier, GradientBoostingClassifier,
    AdaBoostClassifier, ExtraTreesClassifier, IsolationForest)
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, OneClassSVM
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier, LocalOutlierFactor
from sklearn.covariance import EllipticEnvelope

# Optional imports
try:
    from xgboost import XGBClassifier
except: XGBClassifier = None
try:
    from lightgbm import LGBMClassifier
except: LGBMClassifier = None
try:
    from imblearn.over_sampling import SMOTE
    from imblearn.combine import SMOTETomek
    SMOTE_AVAILABLE = True
except: SMOTE_AVAILABLE = False
try:
    from statsmodels.stats.outliers_influence import variance_inflation_factor
    VIF_AVAILABLE = True
except: VIF_AVAILABLE = False

# Evaluation Metrics
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score, roc_curve, 
    precision_recall_curve, average_precision_score, matthews_corrcoef,
    balanced_accuracy_score)

# Feature Selection
from sklearn.feature_selection import (SelectKBest, f_classif, mutual_info_classif,
    RFE, SelectFromModel, VarianceThreshold)
from sklearn.decomposition import PCA

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

# Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
pd.set_option('display.max_columns', 50)
try: plt.style.use('seaborn-v0_8-whitegrid')
except: 
    try: plt.style.use('seaborn-whitegrid')
    except: pass

print("=" * 60)
print("ENVIRONMENT READY")
print(f"Python: {sys.version.split()[0]} | Scikit-learn: {sklearn.__version__}")
print(f"SMOTE Available: {SMOTE_AVAILABLE}")
print("=" * 60)

## 2. Data Loading & Exploration

In [ ]:
# Load Dataset
df = pd.read_excel('Forest Fire Smoke Dataset.xlsx')
print(f"✅ Dataset loaded: {df.shape[0]} rows × {df.shape[1]} columns")
print(f"\nColumns: {df.columns.tolist()}")

# Class Distribution
print("\n" + "=" * 60)
print("CLASS DISTRIBUTION")
print("=" * 60)
class_counts = df['fire_label'].value_counts()
class_pct = df['fire_label'].value_counts(normalize=True) * 100
imbalance_ratio = class_counts[0] / class_counts[1]
print(f"Class 0 (No Fire): {class_counts[0]} ({class_pct[0]:.2f}%)")
print(f"Class 1 (Fire):    {class_counts[1]} ({class_pct[1]:.2f}%)")
print(f"Imbalance Ratio:   {imbalance_ratio:.2f}:1")

# Check if significantly imbalanced (> 2:1)
IS_IMBALANCED = imbalance_ratio > 2
print(f"\n⚠️  Significantly Imbalanced: {IS_IMBALANCED}")
df.head()

In [ ]:
# Comprehensive Descriptive Statistics
print("=" * 60)
print("DESCRIPTIVE STATISTICS")
print("=" * 60)
desc_stats = df.describe().T
desc_stats['skewness'] = df.skew()
desc_stats['kurtosis'] = df.kurtosis()
desc_stats['missing'] = df.isnull().sum()
desc_stats

In [ ]:
# Distribution Visualization
features = df.columns.drop('fire_label').tolist()
n_cols = 3
n_rows = (len(features) + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, n_rows * 3))
axes = axes.flatten()

for i, col in enumerate(features):
    sns.histplot(data=df, x=col, hue='fire_label', kde=True, ax=axes[i], palette='coolwarm')
    axes[i].set_title(f'{col}', fontsize=10)
    axes[i].legend(['No Fire', 'Fire'], fontsize=8)

for j in range(i+1, len(axes)):
    axes[j].set_visible(False)

plt.tight_layout()
plt.suptitle('Feature Distributions by Class', y=1.02, fontsize=14, fontweight='bold')
plt.show()

## 3. Correlation & Multicollinearity Analysis

In [ ]:
# Correlation Matrix
plt.figure(figsize=(12, 10))
corr_matrix = df.corr()
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r', 
            center=0, square=True, linewidths=0.5)
plt.title('Feature Correlation Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# High Correlations
print("\nHIGH CORRELATIONS (|r| > 0.7):")
high_corr = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        if abs(corr_matrix.iloc[i, j]) > 0.7:
            high_corr.append((corr_matrix.columns[i], corr_matrix.columns[j], corr_matrix.iloc[i, j]))
if high_corr:
    for f1, f2, r in high_corr:
        print(f"  {f1} ↔ {f2}: r = {r:.3f}")
else:
    print("  No high correlations found.")

In [ ]:
# Variance Inflation Factor (VIF) for Multicollinearity
print("=" * 60)
print("VARIANCE INFLATION FACTOR (VIF) ANALYSIS")
print("=" * 60)

X_features = df.drop('fire_label', axis=1)

if VIF_AVAILABLE:
    X_scaled = StandardScaler().fit_transform(X_features)
    vif_data = pd.DataFrame()
    vif_data['Feature'] = X_features.columns
    vif_data['VIF'] = [variance_inflation_factor(X_scaled, i) for i in range(X_scaled.shape[1])]
    vif_data = vif_data.sort_values('VIF', ascending=False)
    print(vif_data.to_string(index=False))
    print("\n⚠️  VIF > 10 indicates high multicollinearity")
    
    # Flag high VIF features
    high_vif = vif_data[vif_data['VIF'] > 10]['Feature'].tolist()
    if high_vif:
        print(f"\n🚨 High VIF features: {high_vif}")
else:
    print("statsmodels not available; VIF analysis skipped.")

## 4. Statistical Significance Testing

In [ ]:
# Statistical Tests
print("=" * 60)
print("FEATURE SIGNIFICANCE TESTS")
print("=" * 60)

features = df.columns.drop('fire_label').tolist()
results_sig = []

for col in features:
    fire = df[df['fire_label'] == 1][col]
    no_fire = df[df['fire_label'] == 0][col]
    
    _, p_t = ttest_ind(fire, no_fire)
    _, p_mw = mannwhitneyu(fire, no_fire, alternative='two-sided')
    
    # Effect size (Cohen's d)
    pooled_std = np.sqrt(((len(fire)-1)*fire.std()**2 + (len(no_fire)-1)*no_fire.std()**2) / (len(fire)+len(no_fire)-2))
    cohens_d = (fire.mean() - no_fire.mean()) / pooled_std if pooled_std > 0 else 0
    
    results_sig.append({
        'Feature': col,
        'T-Test p': round(p_t, 6),
        'Mann-Whitney p': round(p_mw, 6),
        "Cohen's d": round(cohens_d, 3),
        'Significant': p_mw < 0.05
    })

sig_df = pd.DataFrame(results_sig).sort_values("Cohen's d", ascending=False, key=abs)
print(f"Significant features: {sig_df['Significant'].sum()}/{len(sig_df)}")
sig_df

## 5. Feature Engineering

In [ ]:
# Feature Engineering
print("=" * 60)
print("FEATURE ENGINEERING")
print("=" * 60)
original_cols = df.shape[1]

# Spectral Ratios
df['red_green_ratio'] = df['mean_red'] / (df['mean_green'] + 1e-10)
df['red_blue_ratio_v2'] = df['mean_red'] / (df['mean_blue'] + 1e-10)
df['green_blue_ratio'] = df['mean_green'] / (df['mean_blue'] + 1e-10)

# Color aggregations
color_cols = ['mean_red', 'mean_green', 'mean_blue']
df['color_mean'] = df[color_cols].mean(axis=1)
df['color_std'] = df[color_cols].std(axis=1)
df['color_range'] = df[color_cols].max(axis=1) - df[color_cols].min(axis=1)

# Log transformations
if 'intensity_std' in df.columns:
    df['log_intensity_std'] = np.log1p(df['intensity_std'].clip(lower=0))
if 'hot_pixel_fraction' in df.columns:
    df['log_hot_pixel'] = np.log1p(df['hot_pixel_fraction'])

# Interaction terms
if 'intensity_std' in df.columns and 'hot_pixel_fraction' in df.columns:
    df['intensity_x_hotpixel'] = df['intensity_std'] * df['hot_pixel_fraction']

# Spatial coords (simulated)
if 'x_coord' not in df.columns:
    df['x_coord'] = np.random.randint(0, 1000, size=len(df))
    df['y_coord'] = np.random.randint(0, 1000, size=len(df))

print(f"✅ Features: {original_cols} → {df.shape[1]} (+{df.shape[1] - original_cols} new)")
print(f"New features: {list(df.columns[original_cols-1:])}")

## 6. Feature Selection

In [ ]:
# Feature Selection
print("=" * 60)
print("FEATURE SELECTION")
print("=" * 60)

X = df.drop('fire_label', axis=1)
y = df['fire_label']

# 1. Variance Threshold
vt = VarianceThreshold(threshold=0.01)
vt.fit(X)
low_var = X.columns[~vt.get_support()].tolist()
print(f"Low variance features removed: {low_var if low_var else 'None'}")

# 2. SelectKBest
k = min(12, X.shape[1])
selector_kb = SelectKBest(score_func=f_classif, k=k)
selector_kb.fit(X, y)
scores = pd.DataFrame({'Feature': X.columns, 'F-Score': selector_kb.scores_})
scores = scores.sort_values('F-Score', ascending=False)
print(f"\nTop {k} features by F-score:")
print(scores.head(k).to_string(index=False))

# 3. Mutual Information
mi_scores = mutual_info_classif(X, y, random_state=SEED)
mi_df = pd.DataFrame({'Feature': X.columns, 'MI-Score': mi_scores}).sort_values('MI-Score', ascending=False)
print(f"\nTop features by Mutual Information:")
print(mi_df.head(8).to_string(index=False))

# 4. RFE with Random Forest
rf_est = RandomForestClassifier(n_estimators=50, random_state=SEED, n_jobs=-1)
rfe = RFE(estimator=rf_est, n_features_to_select=10, step=1)
rfe.fit(X, y)
rfe_features = X.columns[rfe.support_].tolist()
print(f"\nRFE Selected Features (10): {rfe_features}")

---
# 🔀 Dual Approach for Class Imbalance

We implement two strategies:
1. **Approach A (No SMOTE)**: Gaussian/Multivariate Normal Anomaly Detection
2. **Approach B (With SMOTE)**: Traditional Classifiers (RF, GB, AdaBoost, etc.)

---

In [ ]:
# Prepare Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=SEED)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Train: {X_train.shape[0]} | Test: {X_test.shape[0]}")
print(f"Train class distribution: {np.bincount(y_train)}")

## 7A. Approach A: Anomaly Detection (No SMOTE)

Using Gaussian/Multivariate Normal models when not applying SMOTE.

In [ ]:
# Gaussian Anomaly Detection
print("=" * 60)
print("APPROACH A: GAUSSIAN ANOMALY DETECTION (No SMOTE)")
print("=" * 60)

# Train on normal class only (No Fire = 0)
X_train_normal = X_train_scaled[y_train == 0]
print(f"Training on normal class: {X_train_normal.shape[0]} samples")

# 1. Multivariate Gaussian
print("\n🔹 Multivariate Gaussian Model:")
mean = np.mean(X_train_normal, axis=0)
cov = np.cov(X_train_normal, rowvar=False)

# Add regularization to avoid singular matrix
cov += np.eye(cov.shape[0]) * 1e-6

try:
    mvn = multivariate_normal(mean=mean, cov=cov, allow_singular=True)
    log_probs = mvn.logpdf(X_test_scaled)
    
    # Find threshold using training data
    train_log_probs = mvn.logpdf(X_train_scaled)
    threshold = np.percentile(train_log_probs[y_train == 0], 5)  # 5th percentile
    
    y_pred_mvn = (log_probs < threshold).astype(int)
    
    print(f"  Accuracy:  {accuracy_score(y_test, y_pred_mvn):.4f}")
    print(f"  Precision: {precision_score(y_test, y_pred_mvn):.4f}")
    print(f"  Recall:    {recall_score(y_test, y_pred_mvn):.4f}")
    print(f"  F1-Score:  {f1_score(y_test, y_pred_mvn):.4f}")
except Exception as e:
    print(f"  MVN failed: {e}")

# 2. Elliptic Envelope (Robust Gaussian)
print("\n🔹 Elliptic Envelope (Robust Covariance):")
ee = EllipticEnvelope(contamination=0.35, random_state=SEED)
ee.fit(X_train_normal)
y_pred_ee = ee.predict(X_test_scaled)
y_pred_ee = np.where(y_pred_ee == -1, 1, 0)  # Convert: -1 (anomaly) -> 1 (fire)

print(f"  Accuracy:  {accuracy_score(y_test, y_pred_ee):.4f}")
print(f"  Precision: {precision_score(y_test, y_pred_ee):.4f}")
print(f"  Recall:    {recall_score(y_test, y_pred_ee):.4f}")
print(f"  F1-Score:  {f1_score(y_test, y_pred_ee):.4f}")

# 3. Isolation Forest
print("\n🔹 Isolation Forest:")
iso_forest = IsolationForest(contamination=0.35, random_state=SEED, n_jobs=-1)
iso_forest.fit(X_train_normal)
y_pred_iso = iso_forest.predict(X_test_scaled)
y_pred_iso = np.where(y_pred_iso == -1, 1, 0)

print(f"  Accuracy:  {accuracy_score(y_test, y_pred_iso):.4f}")
print(f"  Precision: {precision_score(y_test, y_pred_iso):.4f}")
print(f"  Recall:    {recall_score(y_test, y_pred_iso):.4f}")
print(f"  F1-Score:  {f1_score(y_test, y_pred_iso):.4f}")

# 4. One-Class SVM
print("\n🔹 One-Class SVM:")
ocsvm = OneClassSVM(nu=0.35, kernel='rbf', gamma='auto')
ocsvm.fit(X_train_normal)
y_pred_ocsvm = ocsvm.predict(X_test_scaled)
y_pred_ocsvm = np.where(y_pred_ocsvm == -1, 1, 0)

print(f"  Accuracy:  {accuracy_score(y_test, y_pred_ocsvm):.4f}")
print(f"  Precision: {precision_score(y_test, y_pred_ocsvm):.4f}")
print(f"  Recall:    {recall_score(y_test, y_pred_ocsvm):.4f}")
print(f"  F1-Score:  {f1_score(y_test, y_pred_ocsvm):.4f}")

In [ ]:
# Summary - Approach A
print("\n" + "=" * 60)
print("APPROACH A SUMMARY: ANOMALY DETECTION MODELS")
print("=" * 60)

anomaly_results = []
for name, preds in [('Multivariate Gaussian', y_pred_mvn), 
                     ('Elliptic Envelope', y_pred_ee),
                     ('Isolation Forest', y_pred_iso),
                     ('One-Class SVM', y_pred_ocsvm)]:
    anomaly_results.append({
        'Model': name,
        'Accuracy': round(accuracy_score(y_test, preds), 4),
        'Precision': round(precision_score(y_test, preds), 4),
        'Recall': round(recall_score(y_test, preds), 4),
        'F1-Score': round(f1_score(y_test, preds), 4),
        'MCC': round(matthews_corrcoef(y_test, preds), 4)
    })

anomaly_df = pd.DataFrame(anomaly_results).sort_values('F1-Score', ascending=False)
anomaly_df

## 7B. Approach B: Traditional Classifiers (With SMOTE)

Using SMOTE to balance classes, then applying RF, GB, AdaBoost, etc.

In [ ]:
# Apply SMOTE
print("=" * 60)
print("APPROACH B: SMOTE + TRADITIONAL CLASSIFIERS")
print("=" * 60)

if SMOTE_AVAILABLE:
    smote = SMOTE(random_state=SEED)
    X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)
    print(f"✅ SMOTE Applied")
    print(f"Before: {np.bincount(y_train)} → After: {np.bincount(y_train_smote)}")
else:
    print("⚠️  SMOTE not available. Using class_weight='balanced' instead.")
    X_train_smote, y_train_smote = X_train_scaled, y_train

In [ ]:
# Define Models with balanced class weights
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=SEED, class_weight='balanced'),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=SEED, class_weight='balanced', n_jobs=-1),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=SEED),
    'AdaBoost': AdaBoostClassifier(n_estimators=100, random_state=SEED),
    'Extra Trees': ExtraTreesClassifier(n_estimators=100, random_state=SEED, class_weight='balanced', n_jobs=-1),
    'SVM (RBF)': SVC(probability=True, random_state=SEED, class_weight='balanced'),
    'Decision Tree': DecisionTreeClassifier(random_state=SEED, class_weight='balanced'),
    'Naive Bayes': GaussianNB()
}
if XGBClassifier: 
    models['XGBoost'] = XGBClassifier(n_estimators=100, random_state=SEED, eval_metric='logloss', 
                                       scale_pos_weight=imbalance_ratio, verbosity=0)
if LGBMClassifier: 
    models['LightGBM'] = LGBMClassifier(n_estimators=100, random_state=SEED, class_weight='balanced', verbose=-1)

# Cross-Validation
print("\n" + "=" * 60)
print("CROSS-VALIDATION RESULTS (5-Fold Stratified)")
print("=" * 60)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
cv_results = []

for name, model in models.items():
    scores = cross_validate(model, X_train_smote, y_train_smote, cv=cv,
                           scoring=['accuracy', 'f1', 'roc_auc', 'precision', 'recall'],
                           return_train_score=False, n_jobs=-1)
    cv_results.append({
        'Model': name,
        'CV Accuracy': f"{scores['test_accuracy'].mean():.4f} ± {scores['test_accuracy'].std():.4f}",
        'CV F1': f"{scores['test_f1'].mean():.4f} ± {scores['test_f1'].std():.4f}",
        'CV ROC-AUC': f"{scores['test_roc_auc'].mean():.4f} ± {scores['test_roc_auc'].std():.4f}"
    })

cv_df = pd.DataFrame(cv_results)
cv_df

In [ ]:
# Train & Evaluate on Test Set
print("\n" + "=" * 60)
print("TEST SET EVALUATION (SMOTE Models)")
print("=" * 60)

results = []
trained_models = {}

for name, model in models.items():
    model.fit(X_train_smote, y_train_smote)
    trained_models[name] = model
    
    y_pred = model.predict(X_test_scaled)
    y_prob = model.predict_proba(X_test_scaled)[:, 1] if hasattr(model, 'predict_proba') else None
    
    results.append({
        'Model': name,
        'Accuracy': round(accuracy_score(y_test, y_pred), 4),
        'Precision': round(precision_score(y_test, y_pred), 4),
        'Recall': round(recall_score(y_test, y_pred), 4),
        'F1-Score': round(f1_score(y_test, y_pred), 4),
        'ROC-AUC': round(roc_auc_score(y_test, y_prob), 4) if y_prob is not None else None,
        'PR-AUC': round(average_precision_score(y_test, y_prob), 4) if y_prob is not None else None,
        'MCC': round(matthews_corrcoef(y_test, y_pred), 4)
    })

results_df = pd.DataFrame(results).sort_values('F1-Score', ascending=False)
results_df

In [ ]:
# ROC & PR Curves
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ROC Curves
for name, model in trained_models.items():
    if hasattr(model, 'predict_proba'):
        y_prob = model.predict_proba(X_test_scaled)[:, 1]
        fpr, tpr, _ = roc_curve(y_test, y_prob)
        auc = roc_auc_score(y_test, y_prob)
        axes[0].plot(fpr, tpr, label=f'{name} ({auc:.3f})')

axes[0].plot([0, 1], [0, 1], 'k--', alpha=0.5)
axes[0].set_xlabel('False Positive Rate')
axes[0].set_ylabel('True Positive Rate')
axes[0].set_title('ROC Curves (SMOTE Models)', fontweight='bold')
axes[0].legend(loc='lower right', fontsize=7)

# PR Curves
for name, model in trained_models.items():
    if hasattr(model, 'predict_proba'):
        y_prob = model.predict_proba(X_test_scaled)[:, 1]
        precision, recall, _ = precision_recall_curve(y_test, y_prob)
        pr_auc = average_precision_score(y_test, y_prob)
        axes[1].plot(recall, precision, label=f'{name} ({pr_auc:.3f})')

axes[1].set_xlabel('Recall')
axes[1].set_ylabel('Precision')
axes[1].set_title('Precision-Recall Curves (SMOTE Models)', fontweight='bold')
axes[1].legend(loc='lower left', fontsize=7)

plt.tight_layout()
plt.show()

## 8. Hyperparameter Tuning

In [ ]:
# Hyperparameter Tuning for Top Models
print("=" * 60)
print("HYPERPARAMETER TUNING")
print("=" * 60)

# Random Forest Tuning
print("\n🔧 Tuning Random Forest...")
rf_params = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}
rf_search = RandomizedSearchCV(RandomForestClassifier(random_state=SEED, class_weight='balanced', n_jobs=-1),
                               rf_params, n_iter=10, cv=3, scoring='f1', random_state=SEED, n_jobs=-1)
rf_search.fit(X_train_smote, y_train_smote)
print(f"Best RF params: {rf_search.best_params_}")
print(f"Best RF F1 (CV): {rf_search.best_score_:.4f}")

# Gradient Boosting Tuning
print("\n🔧 Tuning Gradient Boosting...")
gb_params = {
    'n_estimators': [100, 150],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.05, 0.1, 0.2]
}
gb_search = RandomizedSearchCV(GradientBoostingClassifier(random_state=SEED),
                               gb_params, n_iter=10, cv=3, scoring='f1', random_state=SEED, n_jobs=-1)
gb_search.fit(X_train_smote, y_train_smote)
print(f"Best GB params: {gb_search.best_params_}")
print(f"Best GB F1 (CV): {gb_search.best_score_:.4f}")

In [ ]:
# Evaluate Best Tuned Models
print("\n" + "=" * 60)
print("TUNED MODEL PERFORMANCE ON TEST SET")
print("=" * 60)

best_models = {
    'Tuned RF': rf_search.best_estimator_,
    'Tuned GB': gb_search.best_estimator_
}

for name, model in best_models.items():
    y_pred = model.predict(X_test_scaled)
    y_prob = model.predict_proba(X_test_scaled)[:, 1]
    
    print(f"\n{name}:")
    print(f"  Accuracy:  {accuracy_score(y_test, y_pred):.4f}")
    print(f"  F1-Score:  {f1_score(y_test, y_pred):.4f}")
    print(f"  ROC-AUC:   {roc_auc_score(y_test, y_prob):.4f}")
    print(f"  PR-AUC:    {average_precision_score(y_test, y_prob):.4f}")
    print(f"\nConfusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

## 9. Approach Comparison

In [ ]:
# Compare Both Approaches
print("=" * 60)
print("COMPARISON: APPROACH A vs APPROACH B")
print("=" * 60)

# Best from Approach A
best_anomaly = anomaly_df.iloc[0]
print(f"\n✨ Best Anomaly Model: {best_anomaly['Model']}")
print(f"   F1-Score: {best_anomaly['F1-Score']:.4f}")

# Best from Approach B
best_classifier = results_df.iloc[0]
print(f"\n✨ Best SMOTE Classifier: {best_classifier['Model']}")
print(f"   F1-Score: {best_classifier['F1-Score']:.4f}")

# Recommendation
print("\n" + "-" * 60)
if best_classifier['F1-Score'] > best_anomaly['F1-Score']:
    print("📊 RECOMMENDATION: Use SMOTE + Traditional Classifiers")
    final_model = trained_models[best_classifier['Model']]
    final_model_name = best_classifier['Model']
else:
    print("📊 RECOMMENDATION: Use Anomaly Detection Approach")
    final_model = iso_forest  # Default to Isolation Forest
    final_model_name = 'Isolation Forest'

## 10. Feature Importance Analysis

In [ ]:
# Feature Importance
best_rf = rf_search.best_estimator_
fi = pd.DataFrame({'Feature': X.columns, 'Importance': best_rf.feature_importances_})
fi = fi.sort_values('Importance', ascending=False)

plt.figure(figsize=(10, 8))
sns.barplot(data=fi.head(15), x='Importance', y='Feature', palette='viridis')
plt.title('Top 15 Features by Importance (Tuned Random Forest)', fontweight='bold')
plt.tight_layout()
plt.show()

print("\nTop 15 Features:")
print(fi.head(15).to_string(index=False))

## 11. Spatial Risk Heatmap

In [ ]:
# Spatial Risk Analysis
def aggregate_by_grid(df, probs, grid_size=100):
    df_copy = df.copy()
    df_copy['risk'] = probs
    df_copy['gx'] = (df_copy['x_coord'] // grid_size).astype(int)
    df_copy['gy'] = (df_copy['y_coord'] // grid_size).astype(int)
    return df_copy.groupby(['gx', 'gy'])['risk'].agg(['mean', 'max', 'count']).reset_index()

probs = best_rf.predict_proba(scaler.transform(X))[:, 1]
grid = aggregate_by_grid(df, probs)

plt.figure(figsize=(12, 10))
pivot = grid.pivot(index='gy', columns='gx', values='mean')
sns.heatmap(pivot, cmap='YlOrRd', cbar_kws={'label': 'Risk Score'})
plt.title('Forest Fire Risk Heatmap', fontsize=14, fontweight='bold')
plt.xlabel('Grid X')
plt.ylabel('Grid Y')
plt.tight_layout()
plt.show()

## 12. Model Export

In [ ]:
# Export
joblib.dump(best_rf, 'fire_detection_model.pkl')
joblib.dump(scaler, 'feature_scaler.pkl')

config = {
    'model': final_model_name,
    'features': X.columns.tolist(),
    'best_params': rf_search.best_params_,
    'seed': SEED,
    'smote_used': SMOTE_AVAILABLE
}
with open('model_config.json', 'w') as f:
    json.dump(config, f, indent=2)

print("✅ Model, scaler, and config exported!")
print("\n" + "=" * 60)
print("PIPELINE COMPLETE!")
print("=" * 60)